In [1]:
import pandas as pd

In [2]:
dataFile='C:/Anaconda3/Notebooks/machine-learning-demo/Recommender/BX-Book-Ratings.csv'

In [3]:
data=pd.read_csv(dataFile,sep=";",encoding='ISO-8859-1',header=0,names=["user","isbn","rating"])

In [4]:
data.head()

,user,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [5]:
bookFile='C:/Anaconda3/Notebooks/machine-learning-demo/Recommender/BX-Books.csv'
books=pd.read_csv(bookFile,sep=";",header=0,error_bad_lines=False, usecols=[0,1,2],index_col=0,names=['isbn',"title","author"],encoding='ISO-8859-1')

In [6]:
books.head()

,title,author
isbn,,
0195153448,Classical Mythology,Mark P. O. Morford
0002005018,Clara Callan,Richard Bruce Wright
0060973129,Decision in Normandy,Carlo D'Este
0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
0393045218,The Mummies of Urumchi,E. J. W. Barber


In [7]:
def bookMeta(isbn):
    title = books.at[isbn,"title"]
    author = books.at[isbn,"author"]
    return title, author
bookMeta("0671027360")

('Angels &amp; Demons', 'Dan Brown')

In [8]:
data = data[data["isbn"].isin(books.index)]

In [9]:
def faveBooks(user,N):
    userRatings = data[data["user"]==user]
    sortedRatings = pd.DataFrame.sort_values(userRatings,['rating'],ascending=[0])[:N] 
    sortedRatings["title"] = sortedRatings["isbn"].apply(bookMeta)
    return sortedRatings

In [10]:
faveBooks(204622,5)

,user,isbn,rating,title
844955,204622,0967560500,10,"(Natural Hormonal Enhancement, Rob Faigin)"
844935,204622,0671027360,10,"(Angels &amp; Demons, Dan Brown)"
844926,204622,0385504209,10,"(The Da Vinci Code, Dan Brown)"
844958,204622,097173660X,9,"(Life After School Explained, Cap &amp; Compass)"
844920,204622,0060935464,9,"(To Kill a Mockingbird, Harper Lee)"


In [11]:
data.shape

(1031175, 3)

In [12]:
usersPerISBN = data.isbn.value_counts()
usersPerISBN.head(10)

0971880107    2502
0316666343    1295
0385504209     883
0060928336     732
0312195516     723
044023722X     647
0142001740     615
067976402X     614
0671027360     586
0446672211     585
Name: isbn, dtype: int64

In [13]:
usersPerISBN.shape

(270170,)

In [14]:
ISBNsPerUser = data.user.value_counts()

In [15]:
ISBNsPerUser.shape

(92107,)

In [16]:
data = data[data["isbn"].isin(usersPerISBN[usersPerISBN>10].index)]

In [17]:
data = data[data["user"].isin(ISBNsPerUser[ISBNsPerUser>10].index)]

In [18]:
from scipy.sparse import coo_matrix
data['user'] = data['user'].astype("category")
data['isbn'] = data['isbn'].astype("category")

R = coo_matrix((data['rating'].astype(float),
                       (data['user'].cat.codes.copy(),
                        data['isbn'].cat.codes.copy())))

In [20]:
data.head()

,user,isbn,rating
31,276762,034544003X,0
33,276762,0380711524,5
34,276762,0451167317,0
89,276798,3423084049,0
97,276798,3548603203,6


In [21]:
R.shape

(10706, 15451)

In [22]:
len(R.data)

405709

In [23]:
R.data[0]

0.0

In [24]:
R.row[0]

10633

In [25]:
R.col[0]

3053

In [26]:
M,N = R.shape
K = 3

In [50]:
import numpy as np
P = np.random.rand(M,K)
Q = np.random.rand(K,N)
P.shape

(10706, 3)

In [28]:
from numpy.linalg import norm

def error(R,P,Q,lamda=0.02):
    ratings = R.data
    rows = R.row
    cols = R.col
    e = 0 
    for ui in range(len(ratings)):
        rui=ratings[ui]
        u = rows[ui]
        i = cols[ui]
        if rui>0:
            e= e + pow(rui-np.dot(P[u,:],Q[:,i]),2)+\
                lamda*(pow(norm(P[u,:]),2)+pow(norm(Q[:,i]),2))
    return e

In [29]:
error(R,P,Q)

7135925.8156788358

In [30]:
rmse = np.sqrt(error(R,P,Q)/len(R.data))

In [31]:
rmse

4.193897792658861

In [35]:
def SGD(R, K, lamda=0.02,steps=10, gamma=0.001):
    
    M,N = R.shape
    P = np.random.rand(M,K)
    Q = np.random.rand(K,N)
    
    rmse = np.sqrt(error(R,P,Q,lamda)/len(R.data))
    print("Initial RMSE: "+str(rmse))
    
    for step in range(steps):
        for ui in range(len(R.data)):
            rui=R.data[ui]
            u = R.row[ui]
            i = R.col[ui]
            if rui>0:
                eui=rui-np.dot(P[u,:],Q[:,i])
                P[u,:]=P[u,:]+gamma*2*(eui*Q[:,i]-lamda*P[u,:])
                Q[:,i]=Q[:,i]+gamma*2*(eui*P[u,:]-lamda*Q[:,i])
        rmse = np.sqrt(error(R,P,Q,lamda)/len(R.data))
        if rmse<0.5:
            break
    print("Final RMSE: "+str(rmse))
    return P,Q

In [37]:
(P,Q)=SGD(R,K=2,gamma=0.0007,lamda=0.01, steps=100)

Initial RMSE: 4.32723912525
Final RMSE: 0.78673900836


In [38]:
P.shape

(10706, 2)

In [41]:
P

array([[ 0.58795081,  1.41246087],
       [ 2.17041569,  1.77747972],
       [ 2.38583972,  2.36016985],
       ..., 
       [ 0.20871123,  0.19451678],
       [ 1.70222143,  2.58049744],
       [ 1.49958004,  1.72892348]])

In [61]:
P[10705]

array([ 0.30778865,  0.89749568,  0.53300755])

In [46]:
Q[1][12]

2.3489836197753204

In [49]:
Q.shape

(2, 15451)

In [63]:
R.data[1]

5.0